In [1]:
import utils 
import os
os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [2]:
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import torch.nn.functional as F
import random
import os
import numpy as np
# Set the random seed for reproducible experiments
random.seed(230)
torch.manual_seed(230)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from tqdm.auto import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## 1. Load Dataset

In [4]:
from datasets import load_dataset

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}
task_name = "sst2"
datasets = load_dataset("glue",task_name)
datasets

Found cached dataset glue (/home/todsavadt/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 1305.69it/s]


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

## 2. Preprocessing

In [5]:
student = "distilroberta-base"
teacher = "textattack/roberta-base-SST-2"

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(student)

In [7]:
# Labels
if task_name is not None:
    is_regression = task_name == "stsb"
    if not is_regression:
        label_list = datasets["train"].features["label"].names
        num_labels = len(label_list)
    else:
        num_labels = 1
else:
    # Trying to have good defaults here, don't hesitate to tweak to your needs.
    is_regression = datasets["train"].features["label"].dtype in ["float32", "float64"]
    if is_regression:
        num_labels = 1
    else:
        # A useful fast method:
        # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.unique
        label_list = datasets["train"].unique("label")
        label_list.sort()  # Let's sort it for determinism
        num_labels = len(label_list)
        
num_labels, is_regression

(2, False)

In [8]:
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer, PretrainedConfig
model_name_or_path = student
config = AutoConfig.from_pretrained(
    model_name_or_path, 
    num_labels=num_labels, 
    finetuning_task=task_name)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path,
    from_tf=bool(".ckpt" in model_name_or_path),
    config=config,
)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight'

In [9]:
label_to_id = None

if (
    model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id
    and task_name is not None
    and not is_regression
):
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if list(sorted(label_name_to_id.keys())) == list(sorted(label_list)):
        label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
        
elif task_name is None and not is_regression:
    label_to_id = {v: i for i, v in enumerate(label_list)}
    
def tokenize_function(examples):
    sentence1_key, sentence2_key = task_to_keys[task_name]
    args = (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
    result = tokenizer(*args, max_length=180, padding="max_length", truncation=True)
    if "label" in examples:
        if label_to_id is not None:
            # Map labels to IDs (not necessary for GLUE tasks)
            result["label"] = [label_to_id[l] for l in examples["label"]]
        else:
            # In all cases, rename the column to labels because the model will expect that.
            result["label"] = examples["label"]
    
    return result

tokenized_datasets = datasets.map(tokenize_function, batched=True)
tokenized_datasets

Loading cached processed dataset at /home/todsavadt/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-ddce3eb30c7f5991.arrow
Loading cached processed dataset at /home/todsavadt/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-b7071d8f4a273937.arrow
Loading cached processed dataset at /home/todsavadt/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f6295d5d5b5292a7.arrow


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [10]:
#Use list comprehension to extract non-None elements from the tuple
elements = [element for element in task_to_keys[task_name] if element is not None]
elements

['sentence']

In [11]:
tokenized_datasets = tokenized_datasets.remove_columns(elements + ["idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [12]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["validation_matched" if task_name == "mnli" else "validation"].shuffle(seed=42) #.select(range(100))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42)

Loading cached shuffled indices for dataset at /home/todsavadt/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-29e16e0eef6a27c5.arrow
Loading cached shuffled indices for dataset at /home/todsavadt/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-b1f20420ff6bc4cb.arrow
Loading cached shuffled indices for dataset at /home/todsavadt/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-051ba530f8f9dec6.arrow


## 3. Dataloaders

In [13]:
from torch.utils.data import DataLoader
per_device_train_batch_size = 16
per_device_eval_batch_size = 16

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=per_device_train_batch_size)
val_dataloader = DataLoader(small_eval_dataset, batch_size=per_device_eval_batch_size)

## 4. Model

In [14]:
# teacher model
teacher_model = AutoModelForSequenceClassification.from_pretrained(
    teacher,
    config=config,
)

# student model
student_model = AutoModelForSequenceClassification.from_pretrained(
    student,
    config=config,
)

student_model = student_model.to(device)
teacher_model = teacher_model.to(device)

Some weights of the model checkpoint at textattack/roberta-base-SST-2 were not used when initializing RobertaForSequenceClassification: ['roberta.encoder.layer.11.attention.output.LayerNorm.bias', 'roberta.encoder.layer.11.attention.self.value.bias', 'roberta.encoder.layer.6.output.dense.weight', 'roberta.encoder.layer.8.attention.output.dense.weight', 'roberta.encoder.layer.9.attention.self.value.weight', 'roberta.encoder.layer.9.attention.output.LayerNorm.weight', 'roberta.encoder.layer.7.attention.output.dense.weight', 'roberta.encoder.layer.10.output.dense.bias', 'roberta.encoder.layer.8.attention.self.key.weight', 'roberta.encoder.layer.6.attention.self.value.bias', 'roberta.encoder.layer.7.attention.output.LayerNorm.bias', 'roberta.encoder.layer.11.intermediate.dense.bias', 'roberta.encoder.layer.7.intermediate.dense.weight', 'roberta.encoder.layer.9.attention.self.key.weight', 'roberta.encoder.layer.9.intermediate.dense.weight', 'roberta.encoder.layer.9.output.LayerNorm.bias', '

## 5. Training

## Optimizer 

In [15]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

## Accelerator

In [16]:
from accelerate import Accelerator

accelerator = Accelerator()

student_model, optimizer, train_dataloader, val_dataloader = accelerator.prepare(
    student_model, optimizer, train_dataloader, val_dataloader
)

In [17]:
from transformers import get_scheduler
import math

gradient_accumulation_steps = 1
num_update_steps_per_epoch = math.ceil(
        len(train_dataloader) / gradient_accumulation_steps
    )

num_train_epochs = 3
max_train_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

total_batch_size = (
        per_device_train_batch_size
        * accelerator.num_processes
        * gradient_accumulation_steps
    )

### Loss Objective 
The Kullback-Leibler divergence loss. For tensors of the same shape $y_{pred}, y_{true}$ where $y_{pred}$ is the input and $y_{true}$ ​ is the target, we define the pointwise KL-divergence as 

$$L(y_{pred}, y_{true}) = y_{pred}\cdot \log \frac{y_{true}}{y_{pred}}  = y_{true} \cdot (\log y_{true} -\log y_{pred})$$

format : torch.nn.KLDivLoss(size_average=None, reduce=None, reduction='mean', log_target=False)
more infomation click [link](https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html)

In [18]:
def loss_fn_kd(student_outputs, labels, teacher_outputs, alpha = 0.9, T = 1):
    """
    Compute the knowledge-distillation (KD) loss given outputs, labels.
    "Hyperparameters": temperature and alpha

    NOTE: the KL Divergence for PyTorch comparing the softmaxs of teacher
    and student expects the input tensor to be log probabilities! See Issue #2
    """
    loss_fn = nn.KLDivLoss(reduction="batchmean")
    
    loss_logits = loss_fn(F.log_softmax(student_outputs.logits/T, dim=-1),
                             F.softmax(teacher_outputs.logits/T, dim=-1) * (T ** 2))
    KD_loss = alpha * student_outputs.loss + (1. - alpha) * loss_logits
    return KD_loss

## Metrics

In [19]:
def accuracy(outputs, labels):
    """
    Compute the accuracy, given the outputs and labels for all images.

    Args:
        outputs: (np.ndarray) output of the model
        labels: (np.ndarray) [0, 1, ..., num_classes-1]

    Returns: (float) accuracy in [0,1]
    """
    # outputs = np.argmax(outputs, axis=1)
    return np.sum(outputs==labels)/float(labels.size)


# maintain all metrics required in this dictionary- these are used in the training and evaluation loops
metrics = {
    'accuracy': accuracy,
    # could add more metrics such as accuracy for each token type
}

class RunningAverage():
    """A simple class that maintains the running average of a quantity
    
    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """
    def __init__(self):
        self.steps = 0
        self.total = 0
    
    def update(self, val):
        self.total += val
        self.steps += 1
    
    def __call__(self):
        return self.total/float(self.steps)

In [20]:
# Defining train_kd & train_and_evaluate_kd functions
def train_kd(student_model, teacher_model, optimizer, loss_fn_kd, train_dataloader, metrics):
    # set model to training mode
    student_model.train()
    teacher_model.eval()
    
    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = RunningAverage()
    
    for step, batch in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        # batch = tuple(t.to(device) for t in batch)

        # compute model output, fetch teacher output, and compute KD loss
        output_batch = student_model(**batch) # output = loss, logits, hidden_states, attentions
        labels_batch = batch['labels']
        
        # get one batch output from teacher_outputs list
        with torch.no_grad():
            output_teacher_batch = teacher_model(**batch)

        loss = loss_fn_kd(output_batch, labels_batch, output_teacher_batch)

        # clear previous gradients, compute gradients of all variables wrt loss
        optimizer.zero_grad()
        loss.backward()

        # performs updates using calculated gradients
        optimizer.step()

        # Evaluate summaries only once in a while
        if step % gradient_accumulation_steps == 0:
            # extract data from torch Variable, move to cpu, convert to numpy arrays            
            output_batch = output_batch.logits.argmax(dim=-1).cpu().numpy()
            labels_batch = labels_batch.cpu().numpy()

            # compute all metrics on this batch
            summary_batch = {metric:metrics[metric](output_batch, labels_batch)
                             for metric in metrics}
            summary_batch['loss'] = loss.item()
            summ.append(summary_batch)

        # update the average loss
        loss_avg.update(loss.item())
        
    # compute mean of all metrics in summary
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())

In [27]:
def evaluate_kd(model, eval_dataloader, metrics):
    # set model to evaluation mode
    model.eval()
    # summary for current eval loop
    summ = []

    # compute metrics over the dataset
    for step, batch in enumerate(eval_dataloader):
        
        # compute model output
        output_batch = model(**batch)
        labels_batch = batch['labels']
        
        loss = 0.0  #force validation loss to zero to reduce computation time

        # extract data from torch Variable, move to cpu, convert to numpy arrays
        output_batch = output_batch.logits.argmax(dim=-1).cpu().numpy()
        labels_batch = labels_batch.cpu().numpy()

        # compute all metrics on this batch
        summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                         for metric in metrics}
        # summary_batch['loss'] = loss.item()
        summary_batch['loss'] = loss
        summ.append(summary_batch)

    # compute mean of all metrics in summary
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]} 
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    print("- Eval metrics : " + metrics_string)
    return metrics_mean

In [28]:
model_dir = './experiments'
os.path.exists(model_dir)

True

In [29]:
def train_and_evaluate_kd(student_model, teacher_model, train_dataloader, val_dataloader, 
                          optimizer, loss_fn_kd, metrics, model_dir, restore_file=None):
    best_val_acc = 0.0
    for epoch in range(num_train_epochs):
        lr_scheduler.step()
        
         # compute number of batches in one epoch (one full pass over the training set)
        train_kd(student_model, teacher_model, optimizer, loss_fn_kd, train_dataloader,
                 metrics)

        # Evaluate for one epoch on validation set
        val_metrics = evaluate_kd(student_model, val_dataloader, metrics)

        val_acc = val_metrics['accuracy']
        is_best = val_acc>=best_val_acc

        # Save weights
        utils.save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict' : optimizer.state_dict()},
                               is_best=is_best,
                               checkpoint=model_dir)

        # If best_eval, best_save_path
        if is_best:
            print("- Found new best accuracy")
            best_val_acc = val_acc

            # Save best val metrics in a json file in the model directory
            best_json_path = os.path.join(model_dir, "metrics_val_best_weights.json")
            utils.save_dict_to_json(val_metrics, best_json_path)

        # Save latest val metrics in a json file in the model directory
        last_json_path = os.path.join(model_dir, "metrics_val_last_weights.json")
        utils.save_dict_to_json(val_metrics, last_json_path)

In [30]:
train_and_evaluate_kd(student_model, teacher_model, train_dataloader, val_dataloader, optimizer,
                       loss_fn_kd, metrics, model_dir, restore_file=None)

100%|██████████| 63/63 [00:04<00:00, 13.40it/s]


- Eval metrics : accuracy: 0.485 ; loss: 0.000
Checkpoint Directory exists! 
- Found new best accuracy


100%|██████████| 63/63 [00:04<00:00, 13.35it/s]


- Eval metrics : accuracy: 0.485 ; loss: 0.000
Checkpoint Directory exists! 
- Found new best accuracy


100%|██████████| 63/63 [00:04<00:00, 13.23it/s]


- Eval metrics : accuracy: 0.485 ; loss: 0.000
Checkpoint Directory exists! 
- Found new best accuracy
